In [1]:
import pandas as pd
import numpy as np
import S2_0_Loading_Data

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.regularizers import L2
from keras.callbacks import EarlyStopping
from tensorflow.keras.applications import Xception, NASNetLarge
import keras
import tensorflow as tf
import S4_0_Helper_Functions

In [2]:
train_x, test_x, train_y, test_y = S2_0_Loading_Data.load_data(image_size=(331, 331))
train_x.shape

(3257, 331, 331, 3)

In [3]:
pretrained_Xce = Xception(weights='imagenet', include_top=False, input_shape=(331,331, 3))

In [4]:
weight_decay = 0.0005
for layer in pretrained_Xce.layers:
    layer.trainable = False
model = Sequential()
model.add(pretrained_Xce)
model.add(Flatten())
model.add(Dense(512, kernel_regularizer=L2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(Dropout(0.5))
model.add(Dense(3))
model.add(Activation('softmax'))

In [5]:
# compile model
METRICS = [
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'), 
      keras.metrics.CategoricalAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
      keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              loss='CategoricalCrossentropy',
              metrics=METRICS)

# view model layers
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 xception (Functional)       (None, 11, 11, 2048)      20861480  
                                                                 
 flatten (Flatten)           (None, 247808)            0         
                                                                 
 dense (Dense)               (None, 512)               126878208 
                                                                 
 activation (Activation)     (None, 512)               0         
                                                                 
 batch_normalization_4 (Batc  (None, 512)              2048      
 hNormalization)                                                 
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                        

In [6]:
callback = EarlyStopping(monitor='val_prc', mode='max',min_delta = 0.01, verbose=1, patience=10,restore_best_weights=True)

model_hist = model.fit(
    train_x,
    train_y,
    validation_data = (test_x, test_y),
    epochs = 1, # increase if you have a good computer
    verbose = 1,
    callbacks = [callback]
)

102/102 [==============================] - 434s 4s/step - loss: 1.1628 - tp: 2480.0000 - fp: 565.0000 - tn: 5949.0000 - fn: 777.0000 - accuracy: 0.7900 - precision: 0.8144 - recall: 0.7614 - auc: 0.9007 - prc: 0.8331 - val_loss: 0.8166 - val_tp: 728.0000 - val_fp: 79.0000 - val_tn: 1551.0000 - val_fn: 87.0000 - val_accuracy: 0.8945 - val_precision: 0.9021 - val_recall: 0.8933 - val_auc: 0.9633 - val_prc: 0.9373


In [7]:
preds = model.predict(
    test_x
)

S4_0_Helper_Functions.getAccuracyMetrics(preds, test_y)

accuracy: 0.894478527607362
recall: 0.894478527607362
precision: 0.894478527607362


In [8]:
keras.backend.clear_session()

In [9]:
pretrained_NAS = NASNetLarge(weights='imagenet', include_top=False, input_shape=(331, 331, 3))

In [10]:
weight_decay = 0.0005
for layer in pretrained_NAS.layers:
    layer.trainable = False
model = Sequential()
model.add(pretrained_NAS)
model.add(Flatten())
model.add(Dense(512, kernel_regularizer=L2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(Dropout(0.5))
model.add(Dense(3))
model.add(Activation('softmax'))

In [11]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss='CategoricalCrossentropy',
    metrics=METRICS
)

# view model layers
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 NASNet (Functional)         (None, 11, 11, 4032)      84916818  
                                                                 
 flatten (Flatten)           (None, 487872)            0         
                                                                 
 dense (Dense)               (None, 512)               249790976 
                                                                 
 activation_260 (Activation)  (None, 512)              0         
                                                                 
 batch_normalization (BatchN  (None, 512)              2048      
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                        

In [12]:
callback = EarlyStopping(monitor='val_prc', mode='max',min_delta = 0.01, verbose=1, patience=10,restore_best_weights=True)

model_hist = model.fit(
    train_x,
    train_y,
    validation_data = (test_x, test_y),
    epochs = 1, # increase if you have a good computer
    verbose = 1,
    callbacks = [callback]
)

102/102 [==============================] - 1231s 12s/step - loss: 1.9556 - tp: 2903.0000 - fp: 808.0000 - tn: 7336.0000 - fn: 1169.0000 - accuracy: 0.7549 - precision: 0.7823 - recall: 0.7129 - auc: 0.8773 - prc: 0.7972 - val_loss: 1.4859 - val_tp: 667.0000 - val_fp: 133.0000 - val_tn: 1497.0000 - val_fn: 148.0000 - val_accuracy: 0.8245 - val_precision: 0.8338 - val_recall: 0.8184 - val_auc: 0.9397 - val_prc: 0.9044


In [13]:
preds = model.predict(
    test_x
)

S4_0_Helper_Functions.getAccuracyMetrics(preds, test_y)

accuracy: 0.8245398773006135
recall: 0.8245398773006135
precision: 0.8245398773006135
